#### Amazinum. Task to minimize max drops. Moisei

- We need a strategy that tries to make solutions to all possible answers the same depth (same number of drops). The way to reduce the worst case is to attempt to make all cases take the same number of drops.
- If the solution lays somewhere in a floor low down, then we have extra-headroom when we need to step by singles, but, as we get higher up the building, we’ve already used drop chances to get there, so there we have less drops left when we have to switch to going floor-by-floor.
- Imagine we drop our first glass from floor n, if it breaks, we can step through the previous (n-1) floors one-by-one. If it doesn’t break, rather than jumping up another n floors, instead we should step up just (n-1) floors (because we have one less drop available if we have to switch to one-by-one floors), so the next floor we should try is floor n + (n-1). Similarly, if this drop does not break, we next need to jump up to floor n + (n-1) + (n-2), then floor n + (n-1) + (n-2) + (n-3)…
- We keep reducing the step by one each time we jump up, until that step-up is just one floor, and get the following equation for a f floor building: n + (n-1) + (n-2) + (n-3) + (n-4) + … + 1 >= f.
- This summation, as many will recognize, is the formula for triangular numbers (which kind of makes sense, since we’re reducing the step by one each drop we make) and can be simplified to: n (n+1) / 2 >= f.

### В мене получився код з двома результатами. При цьому я вибрав 8 стаканів та 2000 поверхів. Перший результат показує число максимальних кидків, а другий показує кроки випробувань, при якому максимальна кількість кидків виходить ще нижча.
#### 1. Я хотів показати кінцевий результат, а також кроки алгоритму. Вийшло що при показі алгоритму, кількість максимальних кидків стаканів ще менша. З нашими даними це число 12 та якщо дивитись на візуалізацію в вигляді дерева, то число 5.
#### 2. В результаті кидання 8-ми стаканів з 2000 поверхів, ми починаємо з 12го поверху. І далі піднімаємось ще 12 разів аж до 2000 поверху.
#### 3. При decision tree, ми починаємо з 1816го поверху, потім 1944го, 1976го і т. д.

In [159]:
Max = 10000
  
def glassDrop(f, n):
    # A 2D table where entry glassFloor[i][j] will represent minimum
    # number of trials needed for i glasses and j floors.
    glassFloor = [[0 for x in range(f + 1)] for x in range(n + 1)]
  
    # We need one trial for one floor and 0 trials for 0 floors
    for i in range(1, n + 1):
        glassFloor[i][1] = 1
        glassFloor[i][0] = 0
  
    # We always need j trials for one glass and j floors.
    for j in range(1, f + 1):
        glassFloor[1][j] = j
  
    # Fill rest of the entries in table using optimal substructure
    # property
    for i in range(2, n + 1):
        for j in range(2, f + 1):
            glassFloor[i][j] = Max
            for x in range(1, j + 1):
                res = 1 + max(glassFloor[i-1][x-1], glassFloor[i][j-x])
                if res < glassFloor[i][j]:
                    glassFloor[i][j] = res
  
    # glassFloor[n][f] holds the result
    return glassFloor[n][f]

def optimal_solution(f, n):
    
    dp = []

    # With no drops, we can do no floors
    dp.append([0 for x in range(n+1)])

    # With one drop and any glasses, we can do 1 floor
    one_drop = [1 for _ in range(n+1)]
    one_drop[0] = 0 # Except no glasses is still no floors
    dp.append(one_drop)

    # Keep adding drops until we have our answer
    # Array element -1 is shorthand for the end of the array.
    while dp[-1][n] < f:
        # 0 floors for 0 glasses.  1 more for one glass
        next_drop = [0, dp[-1][1] + 1]
        for i in range(2, n+1): # Python for 2..glasses
            # The best we can do is drop at floor dp[-1][i-1].
            # If the glass breaks, we can find the answer using that solution.
            # If the glass holds, we can find another dp[-1][i] floors.
            next_drop.append(dp[-1][i] + dp[-1][i-1])
        dp.append(next_drop)

    return dp

# This turns that optimal solution into a decision tree.    
def dp_to_decision_tree(dp, floors, start_floor=None, glasses=None, drops=None):
    # Figure out defaults if needed.
    if start_floor is None:
        start_floor = 0
    if drops is None:
        drops = len(dp) - 1
    if glasses is None:
        glasses = len(dp[0]) - 1

    if floors == start_floor:
        return start_floor
    elif dp[drops][glasses] < floors - start_floor:
        return None

    while floors - start_floor < dp[drops-1][glasses]:
        drops -= 1

    drop_at = start_floor + dp[drops-1][glasses-1]
    if glasses == 1:
        drop_at = start_floor + 1
    if floors < drop_at:
        drop_at = floors
    return [
        drop_at,
        dp_to_decision_tree(dp,  floors,     drop_at,   glasses, drops-1),
        dp_to_decision_tree(dp, drop_at-1, start_floor, glasses-1, drops-1),
        {'glasses': glasses, 'floor_range': (start_floor, floors)}
        ]

# This prints the decision_tree in a human readable format.
def print_decision_tree(tree, indent="", label="start"):
    if tree is None:
        print(f"{indent}{label}: ?")
    elif isinstance(tree, int):
        print(f"{indent}{label}: {tree} found")
    else:
        print(f"{indent}{label}: {tree[0]} {tree[3]}")
        print_decision_tree(tree[1], indent + "    ", label="didn't broke")
        print_decision_tree(tree[2], indent + "    ", label="broke")

# And this calls the previous functions.
def print_optimal_decisions(f, n):
    print_decision_tree(
        dp_to_decision_tree(
            optimal_solution(f, n), f))
    
     # Driver code
if __name__ == "__main__":
    n = 8
    f = 2000
    print("Minimum number of trials in worst case with " + str(n) + " glasses and "
        + str(f) + " floors is " + str(glassDrop(f, n)) + '\n','\nSteps of trials:\n')

    print_optimal_decisions(f, n)

Minimum number of trials in worst case with 8 glasses and 2000 floors is 12
 
Steps of trials:

start: 1816 {'glasses': 8, 'floor_range': (0, 2000)}
    didn't broke: 1944 {'glasses': 8, 'floor_range': (1816, 2000)}
        didn't broke: 1976 {'glasses': 8, 'floor_range': (1944, 2000)}
            didn't broke: 1992 {'glasses': 8, 'floor_range': (1976, 2000)}
                didn't broke: 2000 {'glasses': 8, 'floor_range': (1992, 2000)}
                    didn't broke: 2000 found
                    broke: 1996 {'glasses': 7, 'floor_range': (1992, 1999)}
                        didn't broke: 1998 {'glasses': 7, 'floor_range': (1996, 1999)}
                            didn't broke: 1999 {'glasses': 7, 'floor_range': (1998, 1999)}
                                didn't broke: 1999 found
                                broke: 1998 found
                            broke: 1997 {'glasses': 6, 'floor_range': (1996, 1997)}
                                didn't broke: 1997 found
            